In [1]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def get_ner(sentence):
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "ner"})
    res = []
    for obj in result['sentences'][0]['entitymentions']:
        res.append((obj['text'], obj['ner']))
    return res

In [2]:
get_ner("you owe obama $4 in indonesia")

[('$4', 'MONEY'), ('indonesia', 'COUNTRY')]

In [3]:
import pandas as pd
from tqdm import  tqdm_notebook

def run():
    df = pd.read_csv("dataset/res16_baru.csv")
    sf = pd.DataFrame(columns = ['id','review','target','category','term'])
    count = 0
    all_ner = []
    money_ner = []
    title_ner = []
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        ner = get_ner(df['review'][index])
        if len(ner):
#             print(df['review'][index])
#             print(ner)
#             print()
            for word, tipe in ner:
                if tipe != 'MONEY':
                    all_ner.append(tipe.lower())
                    all_ner.append(word)
                elif tipe == 'MONEY':
                    money_ner.append(tipe.lower())
                    money_ner.append(word)
    return all_ner, money_ner, title_ner
#         for nn in get_nouns(df['review'][index]):
#             new_term.append(nn)
#         if len(new_term) == 0:
#             new_term = '|'.join(get_nouns(df['review'][index]))
#             count += 1
        
#         sf = sf.append({'id': df['id'][index], 
#                     'review': df['review'][index],
#                     'category': df['category'][index],
#                     'term': '|'.join(new_term)
#                    }, ignore_index=True)
#     print(count)

#     sf.to_csv("Results/add_noun/"+ tipe +".csv")
#     sf.to_excel("Results/add_noun/"+ tipe +".xlsx")

In [4]:
new_ner, mon_ner, tit_ner = run()

HBox(children=(IntProgress(value=0, max=358), HTML(value='')))

In [5]:
splitted_ner = []
for ner in new_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
ner_ner = sorted(list(dict.fromkeys(splitted_ner)))

In [6]:
print(ner_ner)

['13', '15', '17', '2', '2nd', '2\xa01/2', '300', '4', '5', '50', '6th', '8pm.', '9:30', 'a', 'ago', 'america', 'avenue', 'baluchi', 'bbe', 'casa', 'chinatown', 'chinese', 'chutney', 'city', 'citysearch', 'co.', 'cobb', 'college', 'companion', 'cosette', 'country', 'current', 'date', 'day', 'dj', 'duration', 'egyptian', 'emilio', 'european', 'evening', 'every', 'farms', 'femme', 'few', 'first', 'flatbush', 'four', 'french', 'frickin', 'friday', 'gigondas', 'hampton', 'he', 'her', 'him', 'hostess', 'hours', 'hudson', 'hutner', 'indian', 'italian', 'ive', 'japanese', 'la', 'last', 'late', 'lobster', 'location', 'manager', 'manhattan', 'misc', 'months', "mother's", 'murray', 'nationality', 'new', 'night', 'nights', 'noon', 'now', 'number', 'once', 'one', 'onion', 'ordinal', 'organization', 'owner', 'paris', 'parisian', 'patroon', 'person', 'philly', 'present', 'raga', 'recently', 'rice', 'river', 'salad', 'saturday', 'saul', 'seasons', 'set', 'she', 'state_or_province', 'suan', 'summer', 

In [7]:
splitted_ner = []
for ner in mon_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())
new_mon = sorted(list(dict.fromkeys(splitted_ner)))

In [8]:
import pandas as pd

def run_filter(tipe, where, ner):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'category','term', 'polarity'])
    for index, concat_terms in enumerate(df['term']):
        new_terms = []
        try:
            terms = concat_terms.split('|')
        except:
            terms = []
        for term in terms:
#             if term == 'time':
#                 print("   ")
            if term not in ner and len(term) > 3:
                new_terms.append(term)
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'category': df['category'][index],
                        'term': '|'.join(new_terms),
                        'polarity': df['polarity'][index]
                       }, ignore_index=True)

    sf.to_csv("Results/Filter/"+ tipe +"-ner.csv")
    sf.to_excel("Results/Filter/"+ tipe +"-ner.xlsx")

In [14]:
run_filter("dp-partial", 'add_noun/', ner_ner)

In [15]:
import string
def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

In [16]:
import pandas as pd

def run_price(tipe, where):
    df = pd.read_csv("Results/" + where + tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','target', 'label','predict', 'term', 'polarity'])
    for index, reviews in enumerate(df['review']):
        tokenized = reviews.lower().replace('  ', ' ').strip().split(' ')
        predict = df['predict'][index]
        if type(predict) == float:
            predict = ''
        ner_word = ''
        flag = True
        for token in tokenized:
            if flag:
                for money in mon_ner:
                    if money in token:
                        if predict == '':
                            predict += 'PRICES'
                        else:
                            predict += '|PRICES'
                        ner_word = token
                        flag = False
                        break
        terms = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
        if ner_word != '':
            terms.append(ner_word)
        print('|'.join(terms))
        sentence = df['review'][index].lower()
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'target': df['target'][index],
                        'label': df['label'][index],
                        'predict': predict,
                        'term': '|'.join(terms),
                        'polarity': df['polarity'][index]
                       }, ignore_index=True)

    sf.to_csv("Results/Categorization/"+ tipe +"-money.csv")
    sf.to_excel("Results/Categorization/"+ tipe +"-money.xlsx")

In [17]:
run_price('dp-partial-ner-glove', 'Categorization/')

list!interesting|wine!interesting|values!many
price!beat
advanatage!-|half!-|price!-|sushi!-|deal!-|saturday!-
menu!worth|prix!worth|fixe!worth|penny!more
food!fresh|$6
service!great|food!great|price!reasonable|beginning!great|evening!service
pizza!-
wine!decent|prices!decent
pizza!-|way!-
toppings!-|$20,
atmosphere!greatest
selection!great|wine!great|value!great|price!great|champagne!price|house!value
dessert!tiny|$8.00...just
drinks!welll
beers!interesting|wines!interesting
price!reasonable|service!poor
sushi!good
roll!huge|spicy!huge|tuna!huge
choices!limited|food!more|$18
price!great|food!great|service!great
price!reasonable|$3-6

menu!typical|restaurant!menu
wines!average|$70+
food!greatest|menu!greatest|price!good|price!greatest|allergies!food|prixe!food|fixe!food|tasting!food
drinks!-|way!-
resturant!cramped|prices!high
prices!cheap|quality!prices
prices!mortal|gods!mortal|years!last|secret!last
everytime!exceptional|food!exceptional|service!food|value!food
place!pricey|food!wor

room!unattractive|clerks!unattractive|style!more|substance!more|folks!style
place!lovely
lawns!beautiful
place!great
service!good
place!ghetto
atmoshere!great|bit!great
crew!best|winnie!best|staff!winnie|staff!crew
place!quiet
staff!pleasant|wait!pleasant|part!most|beautification!wonderful|fun!staff|way!wonderful|way!wonderful|she's-way-cuter-than-me-that-b@#$*
part!best|atmosphere!best|ls!best|bts!atmosphere|night!late
feature!great|terrace!great|suan!great
sum!dim|atmosphere!dim|wait!long|gig!typical|alternative!wait|tables!like|chinatown!sleek
service!friendly|atmosphere!casual
place!better
management!terrible
service!fast
ambience!correct|staff!correct|time!same|place!ambience
place!romantic
service!slow|drawbacks!service|toast!service
place!beautiful
atomosphere!cozy|tables!cozy
service!prompt
service!friendly
service!excellent|lot!liked|setting/atmosphere!liked
vibe!best|owner!friendly|service!fast
bartender!nicest|manhattan!nicest|usha!bartender
space!attractive|redone!attractiv

In [12]:
ate = ['lda', 'lda2vec', 'plsa', 'dp']
sim = ['glove', 'w2v', 'fasttext', 'elmo', 'elmo_ori']

for a in ate:
    for s in sim:
        run_price(a + '-partial-ner-' + s, 'Categorization/')

place|like
food
good|always|outrageously|food
always
good|list|rest|wine
away
share|family
service|price|come
service|always|everything
edible|special
food|average
enjoyed|special
though|could|tasty
place
made|sake|purple|upon|list|list|request|listed
great|good|tempura|share|spicy|rock|appetizer|roll
great|service
food
service|friendly
ambiance
service|taste
like
great|food
service
restaurant|rest|sushi|sushi
fresh|adequately|seemed|sushi|pretty
good|fish
price|well|sushi
service|well|food
prix|fixe|menu|penny|quantity|quality
fresh|food|filled|much|continually|refilled|tasty|$6
great|dish
good|food|like
like
service|sometimes|sometimes
always
bland|spicy|food|like
ambience|pretty|casual|would
idea|view|chart|house
great|great|great|service|price|food|reasonable
edible|dessert|recommend
cosette|path|bistro
restaurant|rest
food|dessert|average|average
ambience|food
pizza|live
pizza|price
good|pizza
meal
great|good|list|always|food|wine
fresh|delicious|dish
great
wife
food|special
good|

ambiance
service
like|make|wait
great|food
service
restaurant
sushi
good
price|well|worth
service|food|well
menu|worth
food|$6
great|dish
good|food|like|wait
like
service|nice
always
food|like
nice|best|would
idea|view|chart|house
service|great|great|great|price|even|food|reasonable
lava|cake|dessert
cosette|path|bistro
restaurant
food
food
pizza|even
price|pizza
good|pizza|even|make
best
great|good|food|wine|menu|always
really|dish|delicious
great|best
wife|shrimp
even|food|staff|menu|special|wait
good|well|atmosphere|special
friendly|staff|well|always|always|never
price|wine|reasonable
service|great|food|decor|delicious
rude
food
place|service|great|food|wine|always
would|wait
great|wine|always|wait
service|great
friendly|always
pizza|like
pizza
toppings|$20,
good
food
food|wait
service|great|friendly
service|great|pizza
wait
block
best
place|food
tuna|gari
great|food|always
great|menu|never
great|price|atmosphere
food
menu
good
tuna
garden|terrace
service|food
great
restaurant
great

In [12]:
'|'.join(['sd'])

'sd'